In [130]:
import time
import pandas as pd
import numpy as np

start = time.perf_counter()
df = pd.read_csv('./dataset.csv')
stop = time.perf_counter()

print(f"Loading time: {stop - start:.4f} seconds.")

Loading time: 24.1662 seconds.


### 1. Remove unnecessary columns and columns with only one unique value

In [131]:
single_unique_columns = [col for col in df.columns if df[col].nunique() == 1]

In [132]:
single_unique_columns

['Bwd.PSH.Flags',
 'Fwd.URG.Flags',
 'Bwd.URG.Flags',
 'CWE.Flag.Count',
 'Fwd.Avg.Bytes.Bulk',
 'Fwd.Avg.Packets.Bulk',
 'Fwd.Avg.Bulk.Rate',
 'Bwd.Avg.Bytes.Bulk',
 'Bwd.Avg.Packets.Bulk',
 'Bwd.Avg.Bulk.Rate',
 'Label']

In [133]:

df.drop(
    columns=['Flow.ID', 'Timestamp'] + single_unique_columns,
    inplace=True
)

In [134]:
df.columns

Index(['Source.IP', 'Source.Port', 'Destination.IP', 'Destination.Port',
       'Protocol', 'Flow.Duration', 'Total.Fwd.Packets',
       'Total.Backward.Packets', 'Total.Length.of.Fwd.Packets',
       'Total.Length.of.Bwd.Packets', 'Fwd.Packet.Length.Max',
       'Fwd.Packet.Length.Min', 'Fwd.Packet.Length.Mean',
       'Fwd.Packet.Length.Std', 'Bwd.Packet.Length.Max',
       'Bwd.Packet.Length.Min', 'Bwd.Packet.Length.Mean',
       'Bwd.Packet.Length.Std', 'Flow.Bytes.s', 'Flow.Packets.s',
       'Flow.IAT.Mean', 'Flow.IAT.Std', 'Flow.IAT.Max', 'Flow.IAT.Min',
       'Fwd.IAT.Total', 'Fwd.IAT.Mean', 'Fwd.IAT.Std', 'Fwd.IAT.Max',
       'Fwd.IAT.Min', 'Bwd.IAT.Total', 'Bwd.IAT.Mean', 'Bwd.IAT.Std',
       'Bwd.IAT.Max', 'Bwd.IAT.Min', 'Fwd.PSH.Flags', 'Fwd.Header.Length',
       'Bwd.Header.Length', 'Fwd.Packets.s', 'Bwd.Packets.s',
       'Min.Packet.Length', 'Max.Packet.Length', 'Packet.Length.Mean',
       'Packet.Length.Std', 'Packet.Length.Variance', 'FIN.Flag.Count',
       'SYN.

In [135]:
df.drop(
    columns=[
        'Active.Min',
        'Idle.Min',
        'Min.Packet.Length',
        'Bwd.IAT.Min',
        'Fwd.IAT.Min',
        'Flow.IAT.Min',
        'Bwd.Packet.Length.Min',
        'Fwd.Packet.Length.Min'
    ], inplace=True
)

In [136]:
df.columns
print(f"\n {len(df.columns)}")


 66


### 2. Frequency Encoding of Source IP address - finally skipped approach in favor of buckets

In [103]:
freq_encoding = (
    df.groupby('Source.IP').size()
) / len(df)

In [46]:
print(type(freq_encoding))
print(freq_encoding.index)
print("-"*20)
print(freq_encoding)

<class 'pandas.core.series.Series'>
Index(['10.120.1.191', '10.130.1.139', '10.130.10.101', '10.130.10.158',
       '10.130.10.176', '10.130.10.191', '10.130.10.215', '10.130.10.232',
       '10.130.12.1', '10.130.12.118',
       ...
       '98.139.199.204', '98.139.199.205', '98.139.21.169', '98.139.225.168',
       '98.139.225.35', '98.139.225.43', '98.142.102.74', '98.158.99.143',
       '98.158.99.149', '99.198.117.213'],
      dtype='object', name='Source.IP', length=6566)
--------------------
Source.IP
10.120.1.191      3.913570e-06
10.130.1.139      2.795407e-07
10.130.10.101     1.397704e-06
10.130.10.158     2.795407e-07
10.130.10.176     2.795407e-07
                      ...     
98.139.225.43     7.547600e-06
98.142.102.74     1.397704e-06
98.158.99.143     1.118163e-06
98.158.99.149     5.590815e-07
99.198.117.213    8.386222e-07
Length: 6566, dtype: float64


In [104]:
df['Source.IP.FreqEnc'] = df['Source.IP'].apply(lambda x: freq_encoding[x])
# freq_encoding: Series -> Source.IP associated with frequency of occurence
df.drop(columns=['Source.IP'], inplace=True)


### 3. Apply One Hot Encoding of the Transport Layer (ISO/OSI Layer 4) Protocol Column
- Protocol column consist on three unique values 

In [137]:
df['Protocol'].value_counts()

6     3572975
17       2684
0        1637
Name: Protocol, dtype: int64

In [138]:
pd.get_dummies(df['Protocol'])
# For each unique value in Protocol column, create separate column and mark value occurence by 1

,0,6,17
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
3577291,0,1,0
3577292,0,1,0
3577293,0,1,0
3577294,0,1,0


In [139]:
_tmp = pd.get_dummies(df['Protocol']).rename(
    columns={
        0:  "Protocol:Unknown",
        6:  "Protocol.TCP",
        17: "Protocol.UDP"
    }
)

In [140]:
_tmp

,Protocol:Unknown,Protocol.TCP,Protocol.UDP
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
3577291,0,1,0
3577292,0,1,0
3577293,0,1,0
3577294,0,1,0


In [141]:
df = pd.concat([df, _tmp], axis=1)

In [142]:
df['Protocol.UDP']

0          0
1          0
2          0
3          0
4          0
          ..
3577291    0
3577292    0
3577293    0
3577294    0
3577295    0
Name: Protocol.UDP, Length: 3577296, dtype: uint8

In [143]:
df['Protocol']

0          6
1          6
2          6
3          6
4          6
          ..
3577291    6
3577292    6
3577293    6
3577294    6
3577295    6
Name: Protocol, Length: 3577296, dtype: int64

In [144]:
df.drop(columns=['Protocol'], inplace=True)

In [145]:
df.head(3)

,Source.IP,Source.Port,Destination.IP,Destination.Port,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,Total.Length.of.Fwd.Packets,Total.Length.of.Bwd.Packets,Fwd.Packet.Length.Max,...,Active.Std,Active.Max,Idle.Mean,Idle.Std,Idle.Max,L7Protocol,ProtocolName,Protocol:Unknown,Protocol.TCP,Protocol.UDP
0,172.19.1.46,52422,10.200.7.7,3128,45523,22,55,132,110414.0,6,...,0.0,0.0,0.0,0.0,0.0,131,HTTP_PROXY,0,1,0
1,10.200.7.7,3128,172.19.1.46,52422,1,2,0,12,0.0,6,...,0.0,0.0,0.0,0.0,0.0,131,HTTP_PROXY,0,1,0
2,50.31.185.39,80,10.200.7.217,38848,1,3,0,674,0.0,337,...,0.0,0.0,0.0,0.0,0.0,7,HTTP,0,1,0


### 4. Normalization of selected columns. Type of operation - Standarization.

In [146]:
def normalize(column):
    # df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
    return (df[column] - df[column].min()) / (df[column].max() - df[column].min())

In [149]:
prevent_from_normalization = [
    'Source.IP', 'Destination.IP', 'Source.Port', 'Destination.Port', 
    'Protocol:Unknown', 'Protocol.TCP', 'Protocol.UDP', 'L7Protocol',
    'ProtocolName.FreqEnc', 'Fwd.PSH.Flags', 'ProtocolName'
]

In [150]:
for col in df.columns:
    if col not in prevent_from_normalization:
        print(f"Working with: {str(col)}")
        df[col] = (df[col] - df[col].min())/(df[col].max() - df[col].min())
        print(f'Normalized: {str(col)}')

Working with: Flow.Duration
Normalized: Flow.Duration
Working with: Total.Fwd.Packets
Normalized: Total.Fwd.Packets
Working with: Total.Backward.Packets
Normalized: Total.Backward.Packets
Working with: Total.Length.of.Fwd.Packets
Normalized: Total.Length.of.Fwd.Packets
Working with: Total.Length.of.Bwd.Packets
Normalized: Total.Length.of.Bwd.Packets
Working with: Fwd.Packet.Length.Max
Normalized: Fwd.Packet.Length.Max
Working with: Fwd.Packet.Length.Mean
Normalized: Fwd.Packet.Length.Mean
Working with: Fwd.Packet.Length.Std
Normalized: Fwd.Packet.Length.Std
Working with: Bwd.Packet.Length.Max
Normalized: Bwd.Packet.Length.Max
Working with: Bwd.Packet.Length.Mean
Normalized: Bwd.Packet.Length.Mean
Working with: Bwd.Packet.Length.Std
Normalized: Bwd.Packet.Length.Std
Working with: Flow.Bytes.s
Normalized: Flow.Bytes.s
Working with: Flow.Packets.s
Normalized: Flow.Packets.s
Working with: Flow.IAT.Mean
Normalized: Flow.IAT.Mean
Working with: Flow.IAT.Std
Normalized: Flow.IAT.Std
Working wi

In [151]:
df

,Source.IP,Source.Port,Destination.IP,Destination.Port,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,Total.Length.of.Fwd.Packets,Total.Length.of.Bwd.Packets,Fwd.Packet.Length.Max,...,Active.Std,Active.Max,Idle.Mean,Idle.Std,Idle.Max,L7Protocol,ProtocolName,Protocol:Unknown,Protocol.TCP,Protocol.UDP
0,172.19.1.46,52422,10.200.7.7,3128,3.793500e-04,0.000046,0.000101,1.946835e-07,0.000082,0.000183,...,0.0,0.0,0.0,0.0,0.0,131,HTTP_PROXY,0,1,0
1,10.200.7.7,3128,172.19.1.46,52422,0.000000e+00,0.000002,0.000000,1.769850e-08,0.000000,0.000183,...,0.0,0.0,0.0,0.0,0.0,131,HTTP_PROXY,0,1,0
2,50.31.185.39,80,10.200.7.217,38848,0.000000e+00,0.000004,0.000000,9.940657e-07,0.000000,0.010264,...,0.0,0.0,0.0,0.0,0.0,7,HTTP,0,1,0
3,50.31.185.39,80,10.200.7.217,38848,1.800000e-06,0.000000,0.000006,0.000000e+00,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,7,HTTP,0,1,0
4,192.168.72.43,55961,10.200.7.7,3128,6.505583e-04,0.000009,0.000000,1.586965e-06,0.000000,0.016112,...,0.0,0.0,0.0,0.0,0.0,131,HTTP_PROXY,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577291,98.138.79.73,443,10.200.7.199,42135,1.909017e-02,0.000009,0.000007,8.834501e-07,0.000002,0.018244,...,0.0,0.0,0.0,0.0,0.0,91,SSL,0,1,0
3577292,98.138.79.73,443,10.200.7.217,51546,1.916667e-07,0.000009,0.000000,2.135619e-06,0.000000,0.044103,...,0.0,0.0,0.0,0.0,0.0,91,SSL,0,1,0
3577293,98.138.79.73,443,10.200.7.218,44366,2.159710e-02,0.000011,0.000009,1.772800e-06,0.000003,0.018305,...,0.0,0.0,0.0,0.0,0.0,91,SSL,0,1,0
3577294,98.138.79.73,443,10.200.7.195,52341,2.185350e-02,0.000007,0.000006,9.321210e-07,0.000002,0.010721,...,0.0,0.0,0.0,0.0,0.0,91,SSL,0,1,0


### 5. Frequency encoding of services - ProtocolName Column

In [152]:
df['ProtocolName'].value_counts()

GOOGLE          959110
HTTP            683734
HTTP_PROXY      623210
SSL             404883
HTTP_CONNECT    317526
                 ...  
STARCRAFT            3
QQ                   2
99TAXI               1
SOCKS                1
NFS                  1
Name: ProtocolName, Length: 78, dtype: int64

In [153]:
freq_encoding = (df.groupby(['ProtocolName']).size()) / len(df)

In [154]:
freq_encoding

ProtocolName
99TAXI            2.795407e-07
AMAZON            2.428510e-02
APPLE             2.128703e-03
APPLE_ICLOUD      3.354489e-04
APPLE_ITUNES      3.597689e-04
                      ...     
WHOIS_DAS         2.795407e-06
WIKIPEDIA         5.660700e-04
WINDOWS_UPDATE    9.636049e-03
YAHOO             5.945273e-03
YOUTUBE           4.774025e-02
Length: 78, dtype: float64

In [155]:
df['ProtocolName.FreqEnc'] = df['ProtocolName'].apply(lambda x: freq_encoding[x])
df.drop(columns=['ProtocolName'], inplace=True)

### 6. Inspect datatypes

In [156]:
df.dtypes.value_counts()

float64    59
int64       4
uint8       3
object      2
dtype: int64

In [157]:
list(df.select_dtypes(include=['object']).columns)

['Source.IP', 'Destination.IP']

### 7. Assign Source.IP, Destination.IP, Source.Port and Destination.Port to buckets
- get 100 most popular values - consider them as not suspected, assign value of 0
- in case of first occurence, consider as suspected, assign value of 1
- occurences less popular than first 100 entries, assign value of 0.5

In [164]:
source_IP_most = df['Source.IP'].value_counts()[:100].sort_values(ascending=False).index.tolist()
source_IP_all = df['Source.IP'].unique().tolist()

dest_IP_most = df['Destination.IP'].value_counts()[:100].sort_values(ascending=False).index.tolist()
dest_IP_all = df['Destination.IP'].unique().tolist()

source_Port_most = df['Source.Port'].value_counts()[:100].sort_values(ascending=False).index.tolist()
source_Port_all = df['Source.Port'].unique().tolist()

dest_Port_most = df['Destination.Port'].value_counts()[:100].sort_values(ascending=False).index.tolist()
dest_port_all = df['Destination.Port'].unique().tolist()

In [165]:
df['Source.IP.Value'] = df['Source.IP'].apply(
    lambda row: 1 if row in source_IP_most else 0.5 if row in source_IP_all else 0
)

In [166]:
df['Destination.IP.Value'] = df['Destination.IP'].apply(
    lambda row: 1 if row in dest_IP_most else 0.5 if row in dest_IP_all else 0
)

In [167]:
df['Source.Port.Value'] = df['Source.Port'].apply(
    lambda row: 1 if row in source_Port_most else 0.5 if row in source_Port_all else 0
)

In [168]:
df['Destination.Port.Value'] = df['Destination.Port'].apply(
    lambda row: 1 if row in dest_Port_most else 0.5 if row in dest_port_all else 0
)

In [169]:
df.drop(columns=['Source.IP', 'Destination.IP', 'Source.Port', 'Destination.Port'], inplace=True)

### 8. Final pruning

In [172]:
df.columns

Index(['Flow.Duration', 'Total.Fwd.Packets', 'Total.Backward.Packets',
       'Total.Length.of.Fwd.Packets', 'Total.Length.of.Bwd.Packets',
       'Fwd.Packet.Length.Max', 'Fwd.Packet.Length.Mean',
       'Fwd.Packet.Length.Std', 'Bwd.Packet.Length.Max',
       'Bwd.Packet.Length.Mean', 'Bwd.Packet.Length.Std', 'Flow.Bytes.s',
       'Flow.Packets.s', 'Flow.IAT.Mean', 'Flow.IAT.Std', 'Flow.IAT.Max',
       'Fwd.IAT.Total', 'Fwd.IAT.Mean', 'Fwd.IAT.Std', 'Fwd.IAT.Max',
       'Bwd.IAT.Total', 'Bwd.IAT.Mean', 'Bwd.IAT.Std', 'Bwd.IAT.Max',
       'Fwd.PSH.Flags', 'Fwd.Header.Length', 'Bwd.Header.Length',
       'Fwd.Packets.s', 'Bwd.Packets.s', 'Max.Packet.Length',
       'Packet.Length.Mean', 'Packet.Length.Std', 'Packet.Length.Variance',
       'FIN.Flag.Count', 'SYN.Flag.Count', 'RST.Flag.Count', 'PSH.Flag.Count',
       'ACK.Flag.Count', 'URG.Flag.Count', 'ECE.Flag.Count', 'Down.Up.Ratio',
       'Average.Packet.Size', 'Avg.Fwd.Segment.Size', 'Avg.Bwd.Segment.Size',
       'Fwd.Header

In [187]:
final_set_of_columns = [
    "Unnamed: 0",
    "Flow.Duration",
    "Total.Fwd.Packets",
    "Total.Backward.Packets",
    "Total.Length.of.Fwd.Packets",
    "Total.Length.of.Bwd.Packets",
    "Fwd.Packet.Length.Max",
    "Fwd.Packet.Length.Mean",
    "Fwd.Packet.Length.Std",
    "Flow.Bytes.s",
    "Flow.IAT.Mean",
    "Flow.IAT.Std",
    "Fwd.IAT.Mean",
    "Fwd.IAT.Std",
    "Fwd.IAT.Max",
    "Bwd.IAT.Mean",
    "Bwd.IAT.Std",
    "Fwd.PSH.Flags",
    "Fwd.Packets.s",
    "Bwd.Packets.s",
    "Max.Packet.Length",
    "Packet.Length.Mean",
    "Packet.Length.Std",
    "FIN.Flag.Count",
    "SYN.Flag.Count",
    "ACK.Flag.Count",
    "URG.Flag.Count",
    "ECE.Flag.Count",
    "Down.Up.Ratio",
    "Subflow.Fwd.Packets",
    "Subflow.Fwd.Bytes",
    "Subflow.Bwd.Packets",
    "Subflow.Bwd.Bytes",
    "Init_Win_bytes_forward",
    "Init_Win_bytes_backward",
    "min_seg_size_forward",
    "Active.Mean",
    "Active.Std",
    "Idle.Mean",
    "Idle.Std",
    "ProtocolName.FreqEnc",
    "Protocol:Unknown",
    "Protocol.TCP",
    "Protocol.UDP",
    "Source.IP.Value",
    "Source.Port.Value",
    "Destination.IP.Value",
    "Destination.Port.Value"
]

In [188]:
print(len(list(set(df.columns) & set(final_set_of_columns))))
print(len(final_set_of_columns))
print(len(df.columns))
print(len(set(df.columns).intersection(set(final_set_of_columns))))

columns_to_drop = []
for column in df.columns:
    if column not in final_set_of_columns:
        columns_to_drop.append(column)

47
48
68
47


In [189]:
columns_to_drop

['Bwd.Packet.Length.Max',
 'Bwd.Packet.Length.Mean',
 'Bwd.Packet.Length.Std',
 'Flow.Packets.s',
 'Flow.IAT.Max',
 'Fwd.IAT.Total',
 'Bwd.IAT.Total',
 'Bwd.IAT.Max',
 'Fwd.Header.Length',
 'Bwd.Header.Length',
 'Packet.Length.Variance',
 'RST.Flag.Count',
 'PSH.Flag.Count',
 'Average.Packet.Size',
 'Avg.Fwd.Segment.Size',
 'Avg.Bwd.Segment.Size',
 'Fwd.Header.Length.1',
 'act_data_pkt_fwd',
 'Active.Max',
 'Idle.Max',
 'L7Protocol']

In [190]:
df.drop(columns=columns_to_drop, inplace=True)

### 9. Quantization and dtype conversion

In [191]:
df.dtypes.value_counts()

float64    43
uint8       3
int64       1
dtype: int64

In [196]:
float64_columns = list(df.select_dtypes(include=['float64']).columns)
for col in float64_columns:
    df[col] = df[col].astype('float32')

In [197]:
df.dtypes.value_counts()

float32    43
uint8       3
int64       1
dtype: int64

In [198]:
df.head()

,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,Total.Length.of.Fwd.Packets,Total.Length.of.Bwd.Packets,Fwd.Packet.Length.Max,Fwd.Packet.Length.Mean,Fwd.Packet.Length.Std,Flow.Bytes.s,Flow.IAT.Mean,...,Idle.Mean,Idle.Std,Protocol:Unknown,Protocol.TCP,Protocol.UDP,ProtocolName.FreqEnc,Source.IP.Value,Destination.IP.Value,Source.Port.Value,Destination.Port.Value
0,0.000379,0.000046,0.000101,1.946835e-07,0.000082,0.000183,0.000374,0.000000,1.686826e-04,4.989890e-06,...,0.0,0.0,0,1,0,0.174213,0.5,1.0,0.5,1.0
1,0.000000,0.000002,0.000000,1.769850e-08,0.000000,0.000183,0.000374,0.000000,8.335649e-04,6.666667e-09,...,0.0,0.0,0,1,0,0.174213,1.0,0.5,1.0,0.5
2,0.000000,0.000004,0.000000,9.940657e-07,0.000000,0.010264,0.013989,0.031253,4.681856e-02,2.500000e-09,...,0.0,0.0,0,1,0,0.191132,0.5,1.0,1.0,0.5
3,0.000002,0.000000,0.000006,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,6.011111e-07,...,0.0,0.0,0,1,0,0.191132,0.5,1.0,1.0,0.5
4,0.000651,0.000009,0.000000,1.586965e-06,0.000000,0.016112,0.013400,0.046014,9.574087e-07,1.626400e-04,...,0.0,0.0,0,1,0,0.174213,1.0,1.0,0.5,1.0


In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3577296 entries, 0 to 3577295
Data columns (total 47 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Flow.Duration                float32
 1   Total.Fwd.Packets            float32
 2   Total.Backward.Packets       float32
 3   Total.Length.of.Fwd.Packets  float32
 4   Total.Length.of.Bwd.Packets  float32
 5   Fwd.Packet.Length.Max        float32
 6   Fwd.Packet.Length.Mean       float32
 7   Fwd.Packet.Length.Std        float32
 8   Flow.Bytes.s                 float32
 9   Flow.IAT.Mean                float32
 10  Flow.IAT.Std                 float32
 11  Fwd.IAT.Mean                 float32
 12  Fwd.IAT.Std                  float32
 13  Fwd.IAT.Max                  float32
 14  Bwd.IAT.Mean                 float32
 15  Bwd.IAT.Std                  float32
 16  Fwd.PSH.Flags                int64  
 17  Fwd.Packets.s                float32
 18  Bwd.Packets.s                float32
 19  

In [200]:
df.to_csv('final_dataset.csv')

In [205]:
!ls -lh | grep .csv

-rw-rw-r--  1 mateusz  staff   1.6G Jul 11 13:54 dataset.csv
-rw-r--r--  1 mateusz  staff   1.3G Jul 12 12:32 final_dataset.csv
